In [1]:
!pip install deap

     -------------------------------------- 114.3/114.3 kB 6.9 MB/s eta 0:00:00


In [2]:
import random
from deap import base, creator, tools, algorithms
import numpy as np

In [3]:
# Define the fitness function (to be minimized)
def fitness(individual):
    x = individual[0]
    y = 0.02 * x + 0.5 * np.sin(1 * x + 0.1) + 0.75 * np.cos(0.25 * x - 0.3)
    return y,

# Define the individual and the population
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, 0, 20)  # Create random numbers between 0 and 20
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Define the evaluation function
toolbox.register("evaluate", fitness)

# Define the mutation operator
def checkBounds(min, max):
    def decorator(func):
        def wrapper(*args, **kargs):
            offspring = func(*args, **kargs)
            for child in offspring:
                for i in range(len(child)):
                    if child[i] > max:
                        child[i] = max
                    elif child[i] < min:
                        child[i] = min
            return offspring
        return wrapper
    return decorator

toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=3, indpb=1)  # Gaussian mutation
toolbox.decorate("mutate", checkBounds(0, 20))  # Keep mutation within bounds
toolbox.register("mate", tools.cxUniform, indpb=0.5)  # indpb is the probability of each attribute to be exchanged
# Define the selection operator
toolbox.register("select", tools.selTournament, tournsize=4)

In [7]:
def main():
    # Create initial population
    pop = toolbox.population(n=8)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # Parameters for the mu+lambda algorithm
    mu = len(pop)
    lambda_ = round(0.5*len(pop))

    pop, log = algorithms.eaMuPlusLambda(pop, toolbox, mu, lambda_, cxpb=0, mutpb=1,  # cxpb is the crossover probability
                                         ngen=12, stats=stats, halloffame=hof, verbose=True)

    return pop, log, hof

pop, log, hof = main()

# Print best individual
print("Best individual is: %s\nwith fitness: %s" % (hof[0], hof[0].fitness))


gen	nevals	avg      	std     	min      	max     
0  	8     	-0.280768	0.422646	-0.857068	0.604022
1  	4     	-0.602412	0.261912	-0.857068	-0.000351593
2  	4     	-0.836464	0.0376358	-0.857068	-0.743563   
3  	4     	-0.857068	0        	-0.857068	-0.857068   
4  	4     	-0.857068	0        	-0.857068	-0.857068   
5  	4     	-0.857068	0        	-0.857068	-0.857068   
6  	4     	-0.857068	0        	-0.857068	-0.857068   
7  	4     	-0.857068	0        	-0.857068	-0.857068   
8  	4     	-0.806475	0.133857 	-0.857068	-0.452324   
9  	4     	-0.857068	0        	-0.857068	-0.857068   
10 	4     	-0.857068	0        	-0.857068	-0.857068   
11 	4     	-0.857068	0        	-0.857068	-0.857068   
12 	4     	-0.857068	0        	-0.857068	-0.857068   
Best individual is: [11.0891575329225]
with fitness: (-0.857068052808809,)
